<a href="https://colab.research.google.com/github/karfly/learning-deep-learning/blob/master/09_transformer/postagging_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# POS-tagging with Transformers

**Part-of-speech (POS) tagging** is the process of marking up a word in a text as corresponding to a particular part of speech (noun, verb, ...), based on both its definition and its context.


<img src="https://raw.githubusercontent.com/karfly/learning-deep-learning/master/09_transformer/static/pos-tagging.png" width=480>

Let's look at some basic examples:


**Example 1.**

$$\begin{align*}
\text{Input} &= [\text{I love my cat}]\\
\text{Output} &= [\text{PRON VERB PRON NOUN}]
\end{align*}$$


**Example 2.**

$$\begin{align*}
\text{Input} &= [\text{august 11 , 2000}]\\
\text{Output} &= [\text{PROPN NUM PUNCT NUM}]
\end{align*}$$


---
At first glance it seems that the POS-tagging problem can be solved by collecting a dictionary `{word : POS-tag}`. But natural language is very complex and usually a word's part of speech strongly depends on the context, like in the following examples: 


**Example 3.**

$$\begin{align*}
\text{Input} &= [\text{Look , there is a } \textbf{bear}]\\
\text{Output} &= [\text{VERB PUNCT DET VERB DET } \textbf{NOUN}]
\end{align*}$$



**Example 4.**

$$\begin{align*}
\text{Input} &= [\text{I can not } \textbf{bear } \text{it anymore}]\\
\text{Output} &= [\text{PROPN VERB ADV } \textbf{VERB } \text{PROPN ADV}]
\end{align*}$$

In the example 3 the word `bear` is a NOUN, but `bear` in the example 4 is a VERB. This can only be derived from the context. Such examples are not rare, on the contrary, they are very common in human languages (e.g. see [homonyms](https://en.wikipedia.org/wiki/Homonym)). Today we'll tackle this problem by using the most successful (so far) deep learning architecture for NLP – [Transformer](https://arxiv.org/abs/1706.03762)!

## Homework plan

In this homework you'll implement and train 2 POS-tagging models: **Conv** (baseline) and **Transformer**. The **main goal** of this homework is to get to know with the **Transformer architecture** and to understand how it works from the inside.

The plan for the homework is as follows:
1. Load and prepare data (you'll use `UDPOS` dataset from [torchtext](https://pytorch.org/text/))
2. Build baseline Conv model (already implemented for you)
3. Implement training pipeline and train/evaluate baseline model
4. Build Transformer (all fun is here!)
5. Train/evaluate Transformer model (here you'll reuse the pipeline from pt. 3)
6. Write a of short report

## Grading (10 points total)
The main metric in this homework is accuracy calculated on the **test** set (please, don't train on test set – it will be easily revealed by the TAs). Also, you are not allowed to change train/val/test data.

The points are allocated to tasks as follows:
 1. **[3 points]** Training and evaluation of baseline Conv model (don't change the model architecture!). The test accuracy must be **> 82.5%**
 2. **[7 points]** Implementation, training and evaluation of Transformer model. The test accuracy must be **> 85%**
 
Your homework submission should contain:
1. Fully functional notebook, which has to reproduce all the reported results by clicking on the "Run all" button. The notebook must contain the filled in report (see the template in the very end of the notebook).
2. Saved weights of the best models `ConvPOSTagger.best.pth` and `TransformerPOSTagger.best.pth` (saving is already implemented for you).
3. The screenshots of accuracy curves from Tensorboard for both models on validation dataset.

## Tips
- Do not start writing code before you have read and understood the theory.
- Read all the text in this notebook (hope, you're reading this).
- Make one change at a time: never change several things at once (unless you are super confident). Train a model, introduce one change, train again.
- Use Tensorboard. If you use [Google Colab](https://colab.research.google.com/) check this [hack](https://stackoverflow.com/a/57791702).
- Pay much attention to accuracy and loss curves (e.g. in Tensorboard). Track failures early, stop bad experiments early.
- Use GPU. If you don't have it, use [Google Colab](https://colab.research.google.com/).

 
## Disclaimer
ConvNets can be better than Transformers in POS-tagging task in the case of small data. In this homework you'll train models on a very small dataset (12 543 train sentences) which is ridiculous on the scale of modern NLP datasets (millions or even billions of sentences!). Anyway, it's very important to understand how Transformers work and to be ready to apply them in a real-world problem with a large dataset, where Transformers outperform other architectures (Conv, RNN, ...) with a large margin.

**Good luck and have fun!**

## Imports, setup and handy functions
If you don't have some library, just install it with `pip install ...`

In [1]:
# !pip install torchtext

In [2]:
import os
import random
from datetime import datetime

import numpy as np

import torch
from torch import nn
from torch.utils.tensorboard import SummaryWriter

import torchtext
import torchtext.datasets
import torchtext.data

Setup constants:

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

BATCH_SIZE = 64
MAX_LENGTH = 200  # maximum length of the input sentence
MIN_TOKEN_FREQUENCY = 10  # minimum occurrence frequency
DO_TRAIN_CONV_POSTAGGER = True  # set False not to train Conv POS-tagger TODO

Device: cuda


Make experiments reproducible. Set random seed for `random`, `numpy`, `torch` and `cuda`/`cudnn`.

In [4]:
seed = 0

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

torch.backends.cudnn.deterministic = True

Handy function for setting up experiments with Tensorboard logging. By default all the Tensorboard logs will be stored in `./tb` dir.

In [5]:
def setup_experiment(title, logdir="./tb"):
    experiment_name = "{}@{}".format(title, datetime.now().strftime("%d.%m.%Y-%H:%M:%S"))
    writer = SummaryWriter(log_dir=os.path.join(logdir, experiment_name))
    best_model_path = f"{title}.best.pth"
    return writer, experiment_name, best_model_path

Model parameters counter:

In [6]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

## Load and prepare data

In this homework you'll use `UDPOS` dataset for POS-tagging. In fact, `UDPOS` dataset is just a collection of pairs `(sequence of words : sequence of corresponding POS-tags)`.


For easy access to the dataset, use [torchtext](https://pytorch.org/text/) library. At first glance, [torchtext](https://pytorch.org/text/)'s interfaces look very strange, but after multiple uses you get used to it, and it appears to be a very nice library. Let's define `fields` ([torchtext](https://pytorch.org/text/) abstraction) for text and POS-tags:

In [7]:
TEXT = torchtext.data.Field(
    batch_first=True,
    lower=True,  # make lowercase
    init_token='<bos>', eos_token='<eos>'  # special tokens: beginning/end of sequence
)

POSTAG = torchtext.data.Field(
    batch_first=True,
    init_token='<bos>', eos_token='<eos>'
)

Now let's get train/val/test data using defined fields:

In [8]:
train_data, val_data, test_data = torchtext.datasets.UDPOS.splits(
    fields=(('text', TEXT), ('postag', POSTAG))
)

print(f"Train size: {len(train_data)} sequences")
print(f"Val size: {len(val_data)} sequences")
print(f"Test size: {len(test_data)} sequences")

Train size: 12543 sequences
Val size: 2002 sequences
Test size: 2077 sequences


Then we build the vocabulary (*note:* using only train data) to get the mapping from token to some unique index:

In [9]:
TEXT.build_vocab(train_data.text, min_freq=MIN_TOKEN_FREQUENCY)  # filter out rarely occured tokens
POSTAG.build_vocab(train_data.postag)

To convert from token to unique index use `.vocab.stoi[...]` method (`.vocab.itos[...]` for inverse mapping). Here we collect unique indices of `padding` special tokens to use them later:

In [10]:
TEXT_PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
POSTAG_PAD_IDX = POSTAG.vocab.stoi[POSTAG.pad_token]
print(f"TEXT_PAD_IDX={TEXT_PAD_IDX}, POSTAG_PAD_IDX={POSTAG_PAD_IDX}")

TEXT_PAD_IDX=1, POSTAG_PAD_IDX=1


Get vocabulary sizes:

In [11]:
INPUT_DIM = len(TEXT.vocab)
OUTPUT_DIM = len(POSTAG.vocab)
print(f"Number of unique words: {INPUT_DIM}")
print(f"Number of unique POS-tags: {OUTPUT_DIM}")
print("All POS-tags:", ' '.join(POSTAG.vocab.freqs.keys()))

Number of unique words: 2139
Number of unique POS-tags: 21
All POS-tags: PROPN PUNCT ADJ NOUN VERB DET ADP AUX PRON PART SCONJ NUM ADV CCONJ X INTJ SYM


Finally, we define the data iterators that we'll use for training and evaluation:

In [12]:
train_iterator, val_iterator, test_iterator = torchtext.data.BucketIterator.splits(
    (train_data, val_data, test_data), 
     batch_size=BATCH_SIZE,
     device=device
)

Let's grab a batch and look inside:

In [13]:
batch = next(iter(train_iterator))
text, postag = batch.text, batch.postag

print(f"text.shape={text.shape} – [bs, seq_len]")
print(f"postag.shape={postag.shape} – [bs, seq_len]")

text.shape=torch.Size([64, 87]) – [bs, seq_len]
postag.shape=torch.Size([64, 87]) – [bs, seq_len]


Well, that's all about the data!

## Conv POS-tagger (baseline)

Before building the baseline model, let's first formalize our task.

POS-tagging is a "sequence-to-sequence" task, where input and output have the **same length**. Unlike, e.g. machine translation, where the output (translation) can be of any size, in POS-tagging input and output are **aligned**. The number of POS-tags is fixed and not large. So, formally speaking, POS-tagging is a classification problem, but the length of the input can vary.

ConvNets are really good at this kind of task. So, let's implement some simple model consisting of Conv1d layers (with kernels of size 1 => no context is used). Here's the principal scheme:

![](https://raw.githubusercontent.com/karfly/learning-deep-learning/master/09_transformer/static/conv-postagger.png)

The main steps are as follows:
1. Get embeddings of the input sequence
2. Process the embedded sequence with $N$ Conv1d layers. The number of output channels of the final conv layer equals to the the number of unique POS-tags (`output_dim`)

Here's the code:

In [14]:
class ConvPOSTagger(nn.Module):
    def __init__(self, n_tokens, hid_dim, output_dim):
        """ 
        x -> Embedding -> Conv -> ReLU -> Conv -> ReLU -> Conv -> ReLU -> Conv
        
        n_tokens: # of unique tokens
        hid_dim: # of channels of intermediate layers (with expansion)
        output_dim: # of unique POS-tags)
        """
        super().__init__()

        self.emb = nn.Embedding(n_tokens, hid_dim)
        
        # here we use conv layer with kernel size = 1 as a baseline
        # in fact, such model is identical to multiple linear layes applied to each embedded token
        # you can try to increase the kernel size and see how good ConvNets are in POS-tagging when small amount of data is available
        self.layers = nn.Sequential(
            nn.Conv1d(hid_dim, 2 * hid_dim, kernel_size=1),
            nn.ReLU(inplace=True),
            
            nn.Conv1d(2 * hid_dim, 4 * hid_dim, kernel_size=1),
            nn.ReLU(inplace=True),
            
            nn.Conv1d(4 * hid_dim, 8 * hid_dim, kernel_size=1),
            nn.ReLU(inplace=True),
            
            nn.Conv1d(8 * hid_dim, output_dim, kernel_size=1)  # note: no activation here
        )

    def forward(self, x):
        """
        x: torch.long tensor of shape [batch_size, sequence_length]
        returns: torch.float32 tensor of shape [batch_size, sequence_length, output_dim]
        """
        x = self.emb(x)  # get embeddings
        
        # transpose from [batch, time, units] to [batch, units, time] to fit Conv1d dim order
        x = torch.transpose(x, 1, 2)
        
        # encode
        x = self.layers(x)
        
        # transpose back to [batch, time, units]
        x = torch.transpose(x, 1, 2)
        
        return x

Test it:

In [15]:
model = ConvPOSTagger(INPUT_DIM, 64, OUTPUT_DIM).to(device)
dummy_input = batch.text.to(device)  ## your code here
dummy_output = model(dummy_input)

print(f"dummy_output.shape={dummy_output.shape}")

dummy_output.shape=torch.Size([64, 87, 21])


## (3 points) Training pipeline

Any loss function, suitable for classification, will also work for us, so we'll choose the obvious option – [nn.CrossEntropyLoss](https://pytorch.org/docs/stable/nn.html#crossentropyloss) criterion. Make sure to ignore losses calculated over `<pad>` tokens, not to add noise to the total loss.

In [16]:
criterion = nn.CrossEntropyLoss(ignore_index=POSTAG_PAD_IDX)

Next let's setup our model, optimizier and experiment:

In [17]:
model = ConvPOSTagger(INPUT_DIM, 64, OUTPUT_DIM).to(device)
optimizer = torch.optim.Adam(model.parameters())

writer, experiment_name, best_model_path = setup_experiment(model.__class__.__name__, logdir="./tb")
print(f"Experiment name: {experiment_name}")

Experiment name: ConvPOSTagger@24.05.2020-22:15:01


Let's count the total number of models' parameters:

In [18]:
print(f"Model has {count_parameters(model):,} trainable parameters")

Model has 320,597 trainable parameters


A common practice in training deep learning models is to a create single-function `run_epoch(...)`, which can be used both for training and evaluation (see `phase` parameter). Here you need to implement forward of the model and the loss calculation. Good news is that you'll reuse this function later for training the Transformer model.

In [19]:
def run_epoch(model, iterator, optimizer, criterion, phase='train', epoch=0, writer=None):
    is_train = (phase == 'train')
    if is_train:
        model.train()
    else:
        model.eval()
    
    epoch_loss = 0

    # variables for calculating accuracy
    n_predicted = 0
    n_true_predicted = 0
    
    with torch.set_grad_enabled(is_train):
        for i, batch in enumerate(iterator):
            global_i = len(iterator) * epoch + i
            
            # unpack batch
            text, postag = batch.text.to(device), batch.postag.to(device)
            
            # make prediction
            ## your code here
            pred = model(text)
            
            # reshape prediction to [-1, output_dim]
            ## your code here
            pred = pred.reshape(-1, pred.shape[2])
            
            # reshape gt labels to [-1, ]
            ## your code here
            gt = postag.reshape(-1, )
            
            # calculate loss
            ## your code here
            loss = criterion(pred, gt)
            
            if is_train:
                # make optimization step
                ## your code here
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
                
            # calculate accuracy
            n_true_predicted += torch.sum((pred.argmax(-1) == gt) * (gt != POSTAG_PAD_IDX)).item()  # don't calculate pad token
            n_predicted += torch.sum(gt != POSTAG_PAD_IDX).item()
            
            # dump train metrics to tensorboard
            if writer is not None and is_train:
                writer.add_scalar(f"loss/{phase}", loss.item(), global_i)
                
            epoch_loss += loss.item()

        # dump epoch metrics to tensorboard
        if writer is not None:
            writer.add_scalar(f"loss_epoch/{phase}", epoch_loss / len(iterator), epoch)
            writer.add_scalar(f"accuracy_epoch/{phase}", n_true_predicted / n_predicted, epoch)

        return epoch_loss / len(iterator), n_true_predicted / n_predicted

And it's time to train the model. All logs are dumped to `Tesorboard`, so go there and enjoy your beautiful loss curves.

In [20]:
n_epochs = 50
if not DO_TRAIN_CONV_POSTAGGER:
    n_epochs = 0

best_val_loss = float('+inf')
for epoch in range(n_epochs):    
    train_loss, train_accuracy = run_epoch(model, train_iterator, optimizer, criterion, phase='train', epoch=epoch, writer=writer)
    val_loss, val_accuracy = run_epoch(model, val_iterator, None, criterion, phase='val', epoch=epoch, writer=writer)
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), best_model_path)
    
    print(f'Epoch: {epoch+1:2}')
    print(f'\tTrain Loss: {train_loss:.3f} | Train accuracy: {train_accuracy * 100:.2f}')
    print(f'\t Val. Loss: {val_loss:.3f} |  Val. accuracy: {val_accuracy * 100:.2f}') 

Epoch:  1
	Train Loss: 0.997 | Train accuracy: 67.51
	 Val. Loss: 0.591 |  Val. accuracy: 79.44
Epoch:  2
	Train Loss: 0.526 | Train accuracy: 81.72
	 Val. Loss: 0.500 |  Val. accuracy: 81.56
Epoch:  3
	Train Loss: 0.463 | Train accuracy: 83.49
	 Val. Loss: 0.485 |  Val. accuracy: 82.68
Epoch:  4
	Train Loss: 0.449 | Train accuracy: 83.72
	 Val. Loss: 0.500 |  Val. accuracy: 82.14
Epoch:  5
	Train Loss: 0.443 | Train accuracy: 83.76
	 Val. Loss: 0.474 |  Val. accuracy: 82.77
Epoch:  6
	Train Loss: 0.438 | Train accuracy: 83.83
	 Val. Loss: 0.481 |  Val. accuracy: 82.64
Epoch:  7
	Train Loss: 0.436 | Train accuracy: 83.84
	 Val. Loss: 0.481 |  Val. accuracy: 82.54
Epoch:  8
	Train Loss: 0.434 | Train accuracy: 83.86
	 Val. Loss: 0.473 |  Val. accuracy: 82.82
Epoch:  9
	Train Loss: 0.432 | Train accuracy: 83.86
	 Val. Loss: 0.479 |  Val. accuracy: 82.39
Epoch: 10
	Train Loss: 0.431 | Train accuracy: 83.89
	 Val. Loss: 0.475 |  Val. accuracy: 82.85
Epoch: 11
	Train Loss: 0.431 | Train acc

Now we can load our best model and evaluate it on the test set:

In [21]:
model.load_state_dict(torch.load(best_model_path))
test_loss, test_accuracy = run_epoch(model, test_iterator, None, criterion, phase='val')
print(f'Test Loss: {test_loss:.3f} | Test accuracy: {test_accuracy:7.3f}')

Test Loss: 0.470 | Test accuracy:   0.830


Next we'll implement functions to infer our model on any given sentence and display the outputs:

In [22]:
def infer_model(sentence, model, device):
    model.eval()
    
    # get tokens
    if isinstance(sentence, str):
        tokens = [token.lower() for token in sentence.split(' ')]  # split if string
    else:
        tokens = [token.lower() for token in sentence]
    
    # add <bos> and <eos> special tokens to the beginning and to the end
    tokens = ['<bos>'] + tokens + ['<eos>']  ## your code here
    
    # convert tokens to indices
    indices = [TEXT.vocab.stoi[token] for token in tokens]  ## your code here
    indices = torch.LongTensor(indices).unsqueeze(0).to(device)

    # make predictions
    with torch.no_grad():
        pred = model(indices.to(device))  ## your code here
        
    # extract pos-tags indices from predictions using .argmax(...)
    pred_postag_indices = pred.argmax(-1).reshape(-1, )  ## your code here
    
    # convert from indices to pos-tags
    pred_postags = [POSTAG.vocab.itos[idx] for idx in pred_postag_indices]  ## your code here 
    
    # cut off <bos> and <eos>
    tokens = tokens[1:-1]
    pred_postags = pred_postags[1:-1]

    return tokens, pred_postags


def print_predictions(tokens, pred_postags, gt_postags=None):
    print("===> Input sentence:", ' '.join(tokens))
    print()
    
    if gt_postags is not None:
        print("Pred. POS-tag\tGT POS-tag\tCorrect?\tToken\n")
        for token, pred_postag, gt_postag in zip(tokens, pred_postags, gt_postags):
            correct = '✔' if pred_postag == gt_postag else '✘'
            print(f"{pred_postag}\t\t{gt_postag}\t\t{correct}\t\t{token}")
    else:
        print("Pred. POS-tag\tToken\n")

        for token, pred_postag in zip(tokens, pred_postags):
            print(f"{pred_postag}\t\t{token}")
    print()

Sentences from the test set:

In [23]:
example_index = np.random.randint(0, len(test_data.examples))

sentence = test_data.examples[example_index].text
gt_postags = test_data.examples[example_index].postag
tokens, pred_postags = infer_model(sentence, model, device)

print_predictions(tokens, pred_postags, gt_postags)

===> Input sentence: kudos to allentown post office staff

Pred. POS-tag	GT POS-tag	Correct?	Token

NOUN		NOUN		✔		kudos
PART		ADP		✘		to
NOUN		PROPN		✘		allentown
NOUN		PROPN		✘		post
NOUN		PROPN		✘		office
NOUN		NOUN		✔		staff



And here we can try to POS-tag our own sentences. **Likely** the baseline model will fail in the third sentence and tag `bear` as a `NOUN`, not `VERB`. Why?

In [24]:
sentence = 'I will definitely pass this homework'
print_predictions(*infer_model(sentence, model, device))

sentence = 'Look , there is a bear'
print_predictions(*infer_model(sentence, model, device))

sentence = 'I can not bear it anymore'
print_predictions(*infer_model(sentence, model, device))

===> Input sentence: i will definitely pass this homework

Pred. POS-tag	Token

PRON		i
AUX		will
ADV		definitely
VERB		pass
DET		this
NOUN		homework

===> Input sentence: look , there is a bear

Pred. POS-tag	Token

VERB		look
PUNCT		,
PRON		there
AUX		is
DET		a
NOUN		bear

===> Input sentence: i can not bear it anymore

Pred. POS-tag	Token

PRON		i
AUX		can
PART		not
NOUN		bear
PRON		it
NOUN		anymore



### Comment
It probably labels `bear` as a noun, because we used kernel size of `1`, so it doesn't take the context into account.

## (7 points) Transformer

In this part you will implement a slightly modified version of the Transformer model from the [Attention is All You Need](https://arxiv.org/abs/1706.03762) paper. Actually, you'll only implement the encoder part, because it's enough for POS-tagging task.

It's **highly recommended** to read [the original Transformer paper](https://arxiv.org/abs/1706.03762) or [this nice article](https://jalammar.github.io/illustrated-transformer/) before starting coding.

The Transformer consists of multiple nested modules like a matryoshka (e.g. `Encoder` consists of `EncoderLayer`, which consists of `MultiHeadAttentionLayer` and `PositionwiseFeedforwardLayer`). In this notebook the implementation of modules is ordered in a [top-down](https://en.wikipedia.org/wiki/Top-down_and_bottom-up_design) manner – first the higher-level modules are implemented (e.g. `Encoder`), that use lower-level modules (e.g. `EncoderLayer`), which are not implemented yet. It's **strongly recommended** to read the notebook to the end to be aware of the code structure, before you start implementing the Transformer.

![](https://raw.githubusercontent.com/karfly/learning-deep-learning/master/09_transformer/static/transformer.png)

Similar to the Conv model, the Transformer does not use any recurrence. It also does not use any convolutional layers. Instead the model is entirely made up of linear layers, **attention mechanisms** and normalizations. 

As of May 2020, Transformers are the dominant architecture in NLP and are used to achieve state-of-the-art results for many tasks and it appears as if they will be for the near future. 


*Note:* in this notebook you'll implement a **learned positional encoding** (in the fashion of [BERT](https://arxiv.org/abs/1810.04805)), not the static one from the [original paper](https://arxiv.org/abs/1706.03762).

### Encoder

The Transformer's encoder attempts to *transform* the entire source sentence, $X = (x_1, ... ,x_n)$, into a sequence of context vectors, $Z = (z_1, ... , z_n)$. So, if our input sequence was 5 tokens long we would have $Z = (z_1, z_2, z_3, z_4, z_5)$. Why do we call this a sequence of context vectors and not a sequence of hidden states? A hidden state at time $t$ in an RNN has only seen tokens $x_t$ and all the tokens before it. However, each context vector here has seen all tokens at all positions within the input sequence.

![](https://raw.githubusercontent.com/karfly/learning-deep-learning/master/09_transformer/static/transformer-encoder.png)

First, the tokens are passed through a standard embedding layer. Next, as the model has no recurrence it has no idea about the order of the tokens within the sequence. We solve this by using a second embedding layer called a *positional embedding layer*. This is a standard embedding layer where the input is not the token itself but the position of the token within the sequence, starting with the first token, the `<bos>` (beginning of sequence) token, in position 0. The position embedding has a "vocabulary" size of `max_length`, which means our model can accept sentences up to `max_length` tokens long.

The original Transformer implementation from the [Attention is All You Need](https://arxiv.org/abs/1706.03762) paper does not learn positional embeddings. Instead it uses a fixed static embedding. Modern Transformer architectures, like [BERT](https://arxiv.org/abs/1810.04805), use positional embeddings instead, so we'll use them.

Next, the token and positional embeddings are elementwise summed together to get a vector which contains information about the token and also its position within the sequence. However, before they are summed, the token embeddings are multiplied by a scaling factor which is $\sqrt{d_{model}}$, where $d_{model}$ is the hidden dimension size, `hid_dim`. This (supposedly) reduces variance in the embeddings and the model is difficult to train reliably without this scaling factor. Dropout is then applied to the combined embeddings.

The combined embeddings are then passed through $N$ *encoder layers* to get $Z$, which is the output and can be used for any downstream task (e.g. POS-tagging).

The mask, `mask`, is simply the same shape as the source sentence but has a value of 1 when the token in the source sentence is not a `<pad>` token and 0 when it is a `<pad>` token. This is used in the encoder layers to mask the multi-head attention mechanisms, which are used to calculate and apply attention over the source sentence, so the model does not pay attention to `<pad>` tokens, which contain no useful information.

In [25]:
class Encoder(nn.Module):
    def __init__(self,
                 input_dim, 
                 hid_dim, 
                 n_layers, 
                 n_heads, 
                 pf_dim,
                 dropout, 
                 padding_index=None,
                 max_length=128):
        super().__init__()
        
        self.padding_index = padding_index  # if None, don't use masking
        
        # embeddings
        self.tok_embedding = nn.Embedding(input_dim, hid_dim)
        self.pos_embedding = nn.Embedding(max_length, hid_dim)
        
        # encoder layers (implemented below)
        self.layers = nn.ModuleList([EncoderLayer(hid_dim, n_heads, pf_dim, dropout) for _ in range(n_layers)])
        
        # dropout is applied after summing up token and positional embeddings
        self.dropout = nn.Dropout(dropout)
        
        # scale parameter
        self.scale = torch.nn.Parameter(torch.sqrt(torch.tensor(hid_dim, dtype=torch.float32)), requires_grad=False)
        
        # custom weight initialization
        self.init_weights()
        
    def init_weights(self):
        for m in self.modules():
            if hasattr(m, 'weight') and m.weight.dim() > 1:
                nn.init.xavier_uniform_(m.weight.data)
        
    def forward(self, x):
        """
        x (batch of token indices): torch.long tensor of shape [bs, seq_len]
        
        returns (encoded sequence): torch.float32 tensor of shape [bs, seq_len, output_dim]
        """
        device = x.device
        bs, seq_len = x.shape[:2]
        
        # compute non-padding mask (use self.padding_index)
        mask = None
        if self.padding_index is not None:
            mask = (x == self.padding_index)  ## your code here
        
        # get token embeddings and scale with self.scale parameter
        ## your code here
#         tok_embedding = torch.mul(self.tok_embedding(x), 1 / self.scale)
        tok_embedding = self.tok_embedding(x) * self.scale
        
        # generate input [0, 1, ..., seq_len - 1] for positional embedder [bs, seq_len]
        ## your code here
        pos_embed_input = torch.arange(seq_len).repeat(bs, 1).to(device)
#         pos_embed_input = torch.tensor([range(seq_len) for _ in range(bs)]).to(device)
        
        # get pos embeddings
        ## your code here
        pos_embedding = self.pos_embedding(pos_embed_input)
        
        # sum up token and positional embeddings
        ## your code here
        # torch.add(tok_embedding, pos_embedding, out=tok_embedding)
        tok_embedding += pos_embedding
        
        # apply dropout
        ## your code here
        out = self.dropout(tok_embedding)
        
        # apply encoder layers one by one; input shape is [bs, seq_len, hid dim]
        ## your code here
        for layer in self.layers:
            out = layer(out, mask)
        
        return out

### Encoder Layer

The encoder layers are where all of the "meat" of the encoder is contained.

![](https://raw.githubusercontent.com/karfly/learning-deep-learning/master/09_transformer/static/transformer-encoder-layer.png)

The encoder layer consists of 2 main blocks:
1. Pass the source sentence and its mask into the *multi-head attention layer*, perform dropout on it, add a residual connection and pass it through a [Layer Normalization](https://arxiv.org/abs/1607.06450) layer.
2. Pass the output of the 1st block through a *position-wise feedforward* layer and then, again, apply dropout, a residual connection and then layer normalization to get the output of this layer which is fed into the next layer (the parameters are not shared between layers)


The multi-head attention layer is used by the encoder layer to attend to the source sentence, i.e. it is calculating and applying attention over itself instead of another sequence, hence we call it *self attention*.

[This article](https://mlexplained.com/2018/01/13/weight-normalization-and-layer-normalization-explained-normalization-in-deep-learning-part-2/) goes into more detail about layer normalization, but the gist is that it normalizes the values of the features, i.e. across the hidden dimension, so each feature has a mean of 0 and a standard deviation of 1. This allows neural networks with a larger number of layers, like the Transformer, to be trained easier.

In [26]:
class EncoderLayer(nn.Module):
    def __init__(self, 
                 hid_dim, 
                 n_heads, 
                 pf_dim,  
                 dropout):
        super().__init__()
        
        # self-attention layer normalization
        self.attention_layer_norm = nn.LayerNorm(hid_dim)
        
        # positionwise feedforward layer normalization
        self.pf_layer_norm = nn.LayerNorm(hid_dim)
        
        # attention layer (implemented below)
        self.attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout)
        
        # positionwise feedforward layer (implemented below)
        self.pf = PositionwiseFeedforwardLayer(hid_dim, pf_dim, dropout)
        
        # dropout is applied to the outputs of the attention and positionwise feedforward layers
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask=None):
        """
        x (sequence of vectors): torch.float32 tensor of shape [bs, seq_len, hid_dim]
        mask (mask of valid elements): torch.bool tensor of shape [bs, seq_len]
        
        returns (processed sequence of vectors): torch.float32 tensor of shape [bs, seq_len, hid_dim]
        """
        ### block 1
        # calculate self-attention + dropout
        ## your code here
        attention_out = self.attention(x, x, x, mask)
        attention_out = self.dropout(attention_out)
        
        # residual (attention) + attention layer norm
        ## your code here
#         attention_out = self.attention_layer_norm(self.torch.add(attention_out, x, attention_out))
        attention_out = self.attention_layer_norm(attention_out + x)
        
        ### block 2
        # calculate positionwise feedforward + dropout
        ## your code here
        pf_out = self.dropout(self.pf(attention_out))
        
        
        # residual (positionwise feedforward) + positionwise feedforward layer norm
        ## your code here
#         self.pf_layer_norm(torch.add(pf_out, attention_out, pf_out))
        out = self.pf_layer_norm(pf_out + attention_out)
        
        return out

### Multi-Head Attention Layer

One of the key, novel concepts introduced by the Transformer paper is the *multi-head attention layer*. 

![](https://raw.githubusercontent.com/karfly/learning-deep-learning/master/09_transformer/static/transformer-attention.png)

Attention can be thought of as *queries*, *keys* and *values* - where the query is used with the key to get an attention vector (usually the output of a *softmax* operation and has all values between 0 and 1 which sum to 1) which is then used to get a weighted sum of the values.

The Transformer uses *scaled dot-product attention*, where the query and the key are combined by taking the dot product between them, then applying the softmax operation, scaling by $d_k$ and finally multiplying by the value. $d_k$ is the *head dimension*, `head_dim`, which we will shortly explain further.

$$ \text{Attention}(Q, K, V) = \text{Softmax} \big( \frac{QK^T}{\sqrt{d_k}} \big)V $$ 

This is similar to standard *dot product attention* but is scaled by $d_k$, which the paper states is used to stop the results of the dot products growing large, causing gradients to become too small.

However, the scaled dot-product attention isn't simply applied to the queries, keys and values. Instead of doing a single attention application the queries, keys and values have their `hid_dim` split into $h$ *heads* and the scaled dot-product attention is calculated over all heads in parallel. This means instead of paying attention to one concept per attention application, we pay attention to $h$. We then re-combine the heads into their `hid_dim` shape, thus each `hid_dim` is potentially paying attention to $h$ different concepts.

$$ \text{MultiHead}(Q, K, V) = \text{Concat}(\text{head}_1,...,\text{head}_h)W^O $$

$$\text{head}_i = \text{Attention}(QW_i^Q, KW_i^K, VW_i^V) $$

$W^O$ is the linear layer applied at the end of the multi-head attention layer, `fc`. $W^Q, W^K, W^V$ are the linear layers `fc_q`, `fc_k` and `fc_v`.

The steps in this module are as follows:
1. Calculate $QW^Q$, $KW^K$ and $VW^V$ with the linear layers, `fc_q`, `fc_k` and `fc_v` to get `Q`, `K` and `V`.
2. Split the `hid_dim` of the query, key and value into `n_heads` (use `.view`) and correctly permute them so they can be multiplied together.
3. Calculate the `energy` (the un-normalized attention) by multiplying `Q` and `K` together and scaling it by the square root of `head_dim`, which is calculated as `hid_dim // n_heads`. Mask the energy not to pay attention over any elements of the sequence we shouldn't.
4. Apply the softmax, dropout and then apply the attention to the value heads, `V`, before combining the `n_heads` together.
5. Finally, multiply the output of step 4 with $W^O$ (represented as `fc_o`). 

Note that in our implementation the lengths of the keys and values are always the same, thus when matrix multiplying the output of the softmax, `attention`, with `V` we will always have valid dimension sizes for matrix multiplication. This multiplication is carried out using `torch.matmul` (or `@` operator) which, when both tensors are >2-dimensional, does a batched matrix multiplication over the last two dimensions of each tensor. This will be a **[query len, key len] x [value len, head dim]** batched matrix multiplication over the batch size and each head which provides the **[batch size, n heads, query len, head dim]** result.

One thing that looks strange at first is that dropout is applied directly to the attention. This means that our attention vector will most probably not sum to 1 and we may pay full attention to a token but the attention over that token is set to 0 by dropout. This is never explained, or even mentioned, in the paper however is used by the [official implementation](https://github.com/tensorflow/tensor2tensor/) and every Transformer implementation since, [including BERT](https://github.com/google-research/bert/).

In [27]:
class MultiHeadAttentionLayer(nn.Module):
    def __init__(self, hid_dim, n_heads, dropout):
        super().__init__()
        
        assert hid_dim % n_heads == 0, "hid_dim must be divisible by n_heads"
        
        self.hid_dim = hid_dim
        self.n_heads = n_heads
        self.head_dim = hid_dim // n_heads
        
        # query, key and value linear networks
        self.fc_q = nn.Linear(hid_dim, hid_dim)
        self.fc_k = nn.Linear(hid_dim, hid_dim)
        self.fc_v = nn.Linear(hid_dim, hid_dim)
        
        # output linear networks
        self.fc_o = nn.Linear(hid_dim, hid_dim)
        
        # dropout is applied to attention
        self.dropout = nn.Dropout(dropout)
        
        # scale parameter
        self.scale = torch.nn.Parameter(torch.sqrt(torch.tensor(self.head_dim, dtype=torch.float32)), requires_grad=False)
        
    def forward(self, query, key, value, mask=None):
        """
        query/key/value (batch of queries/keys/values): torch.float32 tensor of shape [bs, seq_len, hid_dim]
        mask (mask of valid elements): torch.bool tensor of shape [bs, seq_len]
        
        returns (multi-head attention): torch.float32 tensor of shape [bs, seq_len, hid_dim]
        """
        
        bs = query.shape[0]
        
        # calculate Q, K, V using corresponding linear networks
        q, k, v = self.fc_q(query), self.fc_k(key), self.fc_v(value)  # shape is [bs, seq_len, hid_dim]
                
        # prepare Q, K, V for .matmul() or `@` operatortensor(range(seq_len)).to(device)
        # shape is [bs, n_heads, seq_len, head_dim]
        ## your code here
        seq_len = query.shape[1]
#         q, k, v = q.view(bs, self.n_heads, seq_len, self.head_dim), \
#                   k.view(bs, self.n_heads, seq_len, self.head_dim).permute(0, 1, 3, 2), \
#                   v.view(bs, self.n_heads, seq_len, self.head_dim)
        q, k, v = q.view(bs, -1, self.n_heads, self.head_dim).transpose(1,2), \
                  k.view(bs, -1, self.n_heads, self.head_dim).transpose(1,2), \
                  v.view(bs, -1, self.n_heads, self.head_dim).transpose(1,2)
        
        # compute energy using .matmul() or `@` operator (don't forget to scale!)
        # shape is [bs, n_heads, seq_len, seq_len]
#         energy = q @ k / self.scale  ## your code here  
        energy = q @ k.transpose(-2, -1) / self.scale
        
        # apply mask – 1 in mask is a valid element, 0 - not (use .masked_fill())
        if mask is not None:
            # print(energy.shape, mask.shape)  #torch.Size([64, 8, 46, 46]) torch.Size([64, 46])
#             energy *= mask.reshape(mask.shape[0], 1, 1, mask.shape[1])  ## your code here
            energy.masked_fill_(mask.unsqueeze(1).unsqueeze(1), 1e-9)
        
        # apply softmax along the last dim of energy and get the attention weights
        # shape is [bs, n_heads, seq_len, seq_len]
        attention = torch.softmax(energy, dim=-1)  ## your code here
        
        # weight values with calculated attention + dropout (use .matmul() or `@` operator)
        # shape is [bs, n_heads, seq_len, head_dim]
        x = self.dropout(attention) @ v  ## your code here
        
        # squash 1 and 4 dims back
        x = x.permute(0, 2, 1, 3).contiguous()
        x = x.view(bs, -1, self.hid_dim)  # shape is [bs, seq_len, hid_dim]
        
        # apply output linear layer
        x = self.fc_o(x)  ## your code here
        
        return x

### Position-wise Feedforward Layer

The other main block inside the encoder layer is the *position-wise feedforward layer* This is relatively simple compared to the multi-head attention layer. The input is transformed from `hid_dim` to `pf_dim`, where `pf_dim` is usually a lot larger than `hid_dim`. The original Transformer used a `hid_dim` of 512 and a `pf_dim` of 2048. The ReLU activation function and dropout are applied before it is transformed back into a `hid_dim` representation. 

Why is this used? Unfortunately, it is never explained in the paper.

*The bonus*: we implemented this layer for you!

In [28]:
class PositionwiseFeedforwardLayer(nn.Module):
    def __init__(self, hid_dim, pf_dim, dropout):
        super().__init__()
        
        # linear layers
        self.fc_1 = nn.Linear(hid_dim, pf_dim)
        self.fc_2 = nn.Linear(pf_dim, hid_dim)
        
        # dropout is applied after the first layer
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        """
        x (sequence of vectors): torch.float32 tensor of shape [bs, seq_len, hid_dim]
        
        returns (processed sequence of vectors): torch.float32 tensor of shape [bs, seq_len, hid_dim]
        """
        # apply linear layers + dropout
        x = self.dropout(torch.relu(self.fc_1(x)))
        x = self.fc_2(x)
        
        return x

## Transformer POS-tagger

Now we have all the parts of the transformer and we can build a Transformer-based POS-tagger. It consists of Transformer encoder and single linear layer, which predicts the classes of POS-tags:

In [29]:
class TransformerPOSTagger(nn.Module):
    def __init__(self,
                 input_dim,
                 output_dim,
                 hid_dim=64, 
                 n_layers=8, 
                 n_heads=8, 
                 pf_dim=64,
                 dropout=0.1, 
                 padding_index=None,
                 max_length=128):
        super().__init__()
        
        # transformer encoder
        self.encoder = Encoder(
            input_dim,
            hid_dim=hid_dim, 
            n_layers=n_layers, 
            n_heads=n_heads, 
            pf_dim=pf_dim,
            dropout=dropout, 
            padding_index=padding_index,
            max_length=max_length
        )
        
        # linear layer to predict (classify) pos-tags
        self.postag_predictor = nn.Linear(hid_dim, output_dim)
        
    def forward(self, x):
        # apply encoder
        ## your code here
        encoded = self.encoder(x)
        
        # predict postags
        ## your code here
        out = self.postag_predictor(encoded)
        
        return out

## Training the Transformer Model

The training pipeline for the Transformer model is absolutely the same except hyperparameters. We encourage you to tune them and find the best performing model.

In [30]:
hid_dim = 1024  ## your code here
n_layers = 1  ## your code here
n_heads = 16  ## your code here
pf_dim = 2048  ## your code here
dropout = 0.35  ## your code here


model = TransformerPOSTagger(
    INPUT_DIM,
    OUTPUT_DIM,
    hid_dim=hid_dim, 
    n_layers=n_layers, 
    n_heads=n_heads, 
    pf_dim=pf_dim, 
    dropout=dropout, 
    padding_index=TEXT_PAD_IDX,
    max_length=MAX_LENGTH
).to(device)

The learning rate needs to be lower than the default used by Adam, otherwise the learning can be unstable.

In [31]:
criterion = nn.CrossEntropyLoss(ignore_index=POSTAG_PAD_IDX)  ## your code here
optimizer = torch.optim.Adam(model.parameters(), lr=3e-5)  ## your code here

writer, experiment_name, best_model_path = setup_experiment(model.__class__.__name__, logdir="./tb")
print(f"Experiment name: {experiment_name}")

print(f"Model has {count_parameters(model):,} trainable parameters")

Experiment name: TransformerPOSTagger@24.05.2020-22:17:05
Model has 10,816,533 trainable parameters


Train Transformer POS-tagger:

In [32]:
n_epochs = 50 ## your code here

best_val_loss = float('+inf')
for epoch in range(n_epochs):    
    train_loss, train_accuracy = run_epoch(model, train_iterator, optimizer, criterion, phase='train', epoch=epoch, writer=writer)
    val_loss, val_accuracy = run_epoch(model, val_iterator, None, criterion, phase='val', epoch=epoch, writer=writer)
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), best_model_path)
    
    print(f'Epoch: {epoch+1:02}')
    print(f'\tTrain Loss: {train_loss:.3f} | Train accuracy: {train_accuracy * 100:.2f}')
    print(f'\t Val. Loss: {val_loss:.3f} |  Val. accuracy: {val_accuracy * 100:.2f}') 

Epoch: 01
	Train Loss: 1.315 | Train accuracy: 61.21
	 Val. Loss: 0.693 |  Val. accuracy: 77.58
Epoch: 02
	Train Loss: 0.680 | Train accuracy: 78.11
	 Val. Loss: 0.537 |  Val. accuracy: 82.51
Epoch: 03
	Train Loss: 0.542 | Train accuracy: 81.77
	 Val. Loss: 0.497 |  Val. accuracy: 83.40
Epoch: 04
	Train Loss: 0.488 | Train accuracy: 83.03
	 Val. Loss: 0.481 |  Val. accuracy: 83.47
Epoch: 05
	Train Loss: 0.462 | Train accuracy: 83.52
	 Val. Loss: 0.476 |  Val. accuracy: 83.56
Epoch: 06
	Train Loss: 0.445 | Train accuracy: 83.89
	 Val. Loss: 0.463 |  Val. accuracy: 83.85
Epoch: 07
	Train Loss: 0.434 | Train accuracy: 84.18
	 Val. Loss: 0.459 |  Val. accuracy: 83.96
Epoch: 08
	Train Loss: 0.427 | Train accuracy: 84.36
	 Val. Loss: 0.450 |  Val. accuracy: 84.11
Epoch: 09
	Train Loss: 0.419 | Train accuracy: 84.56
	 Val. Loss: 0.450 |  Val. accuracy: 84.20
Epoch: 10
	Train Loss: 0.414 | Train accuracy: 84.66
	 Val. Loss: 0.442 |  Val. accuracy: 84.34
Epoch: 11
	Train Loss: 0.411 | Train acc

Evaluate the final test accuracy and save best model:

In [33]:
model.load_state_dict(torch.load(best_model_path))
test_loss, test_accuracy = run_epoch(model, test_iterator, None, criterion, phase='val')
print(f'Test Loss: {test_loss:.3f} | Test accuracy: {test_accuracy:7.3f}')

Test Loss: 0.423 | Test accuracy:   0.851


Explore how the trained model works on test sentences:

In [34]:
example_index = np.random.randint(0, len(test_data.examples))

sentence = test_data.examples[example_index].text
gt_postags = test_data.examples[example_index].postag
tokens, pred_postags = infer_model(sentence, model, device)

print_predictions(tokens, pred_postags, gt_postags)

===> Input sentence: swetha

Pred. POS-tag	GT POS-tag	Correct?	Token

PROPN		PROPN		✔		swetha



And on our own sentences:

In [35]:
sentence = 'I will definitely pass this homework'
print_predictions(*infer_model(sentence, model, device))

sentence = 'Look , there is a bear'
print_predictions(*infer_model(sentence, model, device))

sentence = 'I can not bear it anymore'
print_predictions(*infer_model(sentence, model, device))

===> Input sentence: i will definitely pass this homework

Pred. POS-tag	Token

PRON		i
AUX		will
ADV		definitely
VERB		pass
DET		this
NOUN		homework

===> Input sentence: look , there is a bear

Pred. POS-tag	Token

NOUN		look
PUNCT		,
PRON		there
VERB		is
DET		a
NOUN		bear

===> Input sentence: i can not bear it anymore

Pred. POS-tag	Token

PRON		i
AUX		can
PART		not
VERB		bear
PRON		it
VERB		anymore



## Report
1. Did you manage to train the baseline Conv model with the test accuracy **> 82.5%**? Choose <font color='green'>YES</font> or  <font color='red'>NO</font>

2. Did you manage to train the Transformer model with the test accuracy **> 85.0%**? Choose <font color='green'>YES</font> or  <font color='red'>NO</font>

3. Did your Trasformer managed to correctly tag `bear` as a `VERB` in sentence **"I can not bear it anymore"**? Choose <font color='green'>YES</font> or  <font color='red'>NO</font>

4. Did you attach to your submission the screenshots of accuracy curves from Tensorboard for both models on validation dataset? Choose <font color='green'>YES</font> or  <font color='red'>NO</font>

5. Did you attach to your submission the model checkpoints: `ConvPOSTagger.best.pth` and `TransformerPOSTagger.best.pth`? Choose <font color='green'>YES</font> or  <font color='red'>NO</font>

6. What was the most difficult part of the homework for you? [Your answer here]

7. What experiments with Transformer did you conduct? What hyperparameters worked the best? [Your answer here]

8. And here you're free to tell us everything you want. [Your answer here]

## Acknowledgements
- Big thanks to Ben Trevett for creating the core part of the Transformer code.

---